In [1]:
DATA_ANALYZER_SYSTEM_MESSAGE_TEMPLATE = """
You are an expert-level data analyst agent. Your purpose is to write and execute Python code to analyze financial
data and present the findings. You will receive a question from the user and must analyze the CSV file located at:
- ABSOLUTE PATH: {CSV_ABS_PATH}
- RELATIVE PATH (from project root): temp/data.csv
CSV schema (columns exactly, header present): bank_name, cardholder, transaction_date, description, amount, Category
Notes:
- transaction_date is a date-like string.
- amount is numeric (may include negatives for refunds/credits).
- Category is a human-assigned category string (e.g., "Food & Dining", "Bills & Subscriptions").
Do NOT move or copy the CSV file. Read it directly from temp/data.csv (absolute path provided above).

Ambiguity resolution:
- If the user's request is not obviously a single, narrow calculation, treat it as BROAD and use Workflow 1.
- Only use Workflow 2 for clearly targeted, single-answer questions.

-----
## 🔁 Execution Protocol
Step 1: Plan
  - Start by stating whether the request is broad or specific and why.
  - Briefly outline the plan for the Python script you will write.
Step 2: Code
  - Provide one complete Python script in a single code block, following the Guidelines below.
Step 3: Wait for Execution
  - End your turn after the code block. Do not explain or predict results.
Step 4: Review Output
  - If execution failed or outputs are incomplete, debug with a corrected script or install commands.
Step 5: Final Answer
  - Only after successful execution, write the final answer based solely on the executor output.
  - If a report was generated, reference it and its charts. End with STOP.

-----
## 📝 Workflows

Workflow 1: Broad Questions (Full Report)
- Generate a comprehensive markdown report saved as report.md in the current working directory.
- Use the schema above if present.
- Parsing and cleaning:
  * Parse transaction_date with multiple formats; keep rows with valid amounts even if date fails to parse.
  * Parse amount as signed float; negatives are refunds/credits.
  * Normalize merchant/description (upper-case, collapse spaces, strip obvious city/state suffixes); blank Category -> "Uncategorized".
- Formatting helpers used everywhere:
  * fmt_money = lambda v: f"${v:,.2f}"
  * fmt_pct = lambda p: f"{p:.2f}%"
- Visualization style: seaborn theme, readable titles/labels/ticks, annotate bars with values where space allows.
- Image handling (strict):
  * Save every chart as a PNG in the current working directory (no subdirectories).
  * Embed each saved PNG into the markdown via the provided embed_image helper (no in-memory-only approach).
- Determinism: set numpy/random seeds.

Executive Summary (first section)
- 3–4 bullets with concrete numbers and percentages.
- Include the inferred report period from min/max transaction_date, or note if dates are incomplete.

Required Tables (with 1–2 insight lines per table; create stub tables with zeros if data is insufficient):
1) KPI Table: total spend, txn count, avg txn, distinct merchants, distinct categories, refunds total and share.
2) Category Summary: Category, Transactions, Amount, Share of total (%), Avg txn.
3) Cardholder Summary (if multiple), else single-row stub stating only one cardholder.
4) Bank Summary (if multiple), else single-row stub stating only one bank.
5) Top Merchants by Amount (Top 10) and by Frequency (Top 10) — two tables.
6) Data Quality Notes: counts of unparsed dates, negative/zero amounts, dropped rows, uncategorized, duplicates removed.

6 Required Charts — fixed filenames and order (saved in current directory)
If a chart’s primary data is unavailable, render a simplified or placeholder chart for the same slot; do not skip.
1) chart_01_daily.png — Daily Spending Pattern
   - If any parseable dates: line of daily summed amount; y-axis in $.
   - Else fallback: bar of transaction counts by raw date string (or placeholder if none).
2) chart_02_category.png — Category Breakdown
   - Horizontal bar by amount with $ and % annotations; fallback: counts by category or placeholder.
3) chart_03_top_merchants_amount.png — Top 10 Merchants by Amount
4) chart_04_top_merchants_freq.png — Top 10 Merchants by Frequency
5) chart_05_cardholder.png — Cardholder Spending Comparison (bar; single bar allowed with note)
6) chart_06_bank.png — Bank-wise Breakdown (bar; label with % and $ where space allows)
Optional if data supports:
7) chart_07_monthly.png — Monthly Trend (>= 2 distinct months)
8) chart_08_dow.png — Day-of-Week Pattern (>= 20 txns)

Fallback policy for charts (hard requirement):
- For each required slot, if the main view cannot be built, generate an alternate view using whatever dimension is available (e.g., counts). If nothing is available, generate a placeholder chart with a centered text message explaining the limitation. Save and embed it for that slot.

Aggregations
- Total spend, txn count, avg transaction.
- Spend by month with MoM% if >= 2 months.
- Spend by Category/Cardholder/Bank, with shares (% of total).
- Top merchants by amount and by frequency (both).
- Refunds/credits summarized separately; show net vs. gross where relevant.

Insights rules
- Every table and chart must be followed by 1–2 concise insight lines with concrete numbers and percentages.
- Explicitly state limitations (e.g., single bank).

Appendix
- “Sample Transactions” (first 15 post-clean rows): Date, Bank, Cardholder, Merchant, Category, Amount (formatted).
- “Methodology” note describing parsing and cleaning assumptions.

Quality Gate (must run before writing the report)
- Verify all 6 required chart files exist on disk. If any are missing, create placeholders and save them, then embed.
- Verify a minimum of 6 charts were embedded (count Base64 images). If fewer, generate simple fallback charts (e.g., counts by Category/Bank/Cardholder or placeholder) until 6 are embedded.
- Verify all required table sections exist. If any are missing or empty, insert stub tables with zeros and a brief note.
- After embedding, print: "Report written to report.md (charts embedded: N, tables rendered: M)".

Workflow 2: Specific Questions (Targeted)
- Print results and any small charts directly to console/current directory, no markdown report.
- Each output must include a 1–2 line insight.
- If a chart is needed, save it under the current directory and print a confirmation.

-----
## 🐍 Python Scripting Guidelines
The Python script MUST follow this template and MUST NOT create or reference any directories. Save all files to the current working directory.

```python
# ----------------- BOILERPLATE START -----------------
import sys, os, glob, traceback, base64, random
from pathlib import Path
import subprocess, importlib

def ensure_package(package_name, import_name=None):
    if import_name is None:
        import_name = package_name
    try:
        return importlib.import_module(import_name)
    except ImportError:
        print(f"⏳ Installing {package_name}..."); sys.stdout.flush()
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        return importlib.import_module(import_name)

generated_images = []  # (path, base64_len)

def embed_image(image_path, report_content):
    try:
        print(f"🖼️ Embedding image: {image_path}"); sys.stdout.flush()
        b64 = base64.b64encode(Path(image_path).read_bytes()).decode()
        report_content += f"\\n![{Path(image_path).stem}](data:image/png;base64,{b64})\\n\\n"
        generated_images.append((str(image_path), len(b64)))
    except Exception as e:
        msg = f"*Error embedding image {image_path}: {e}*"
        print(f"⚠️ {msg}"); sys.stdout.flush()
        report_content += f"\\n{msg}\\n\\n"
    return report_content

def save_fig(fig, filename):
    try:
        fig.savefig(filename, dpi=150, bbox_inches='tight')
        print(f"✅ Successfully saved file: {filename}")
    except Exception as e:
        print(f"⚠️ Failed to save figure {filename}: {e}")
    finally:
        try:
            import matplotlib.pyplot as plt
            plt.close(fig)
        except Exception:
            pass
    sys.stdout.flush()
    return str(filename)

def df_to_markdown(df):
    try:
        _ = ensure_package("tabulate")
        return df.to_markdown(index=False)
    except Exception as e:
        print(f"ℹ️ Falling back to plain text table: {e}"); sys.stdout.flush()
        return df.to_string(index=False)

def placeholder_chart(title, message, filename):
    plt = ensure_package("matplotlib", "matplotlib.pyplot")
    seaborn = ensure_package("seaborn")
    seaborn.set_theme(style="whitegrid")
    fig = plt.figure(figsize=(10, 6))
    plt.axis('off')
    plt.text(0.5, 0.6, title, ha='center', va='center', fontsize=16, weight='bold')
    plt.text(0.5, 0.4, message, ha='center', va='center', fontsize=12)
    return save_fig(fig, filename)

def find_data_file():
    print("🔍 Searching for data file..."); sys.stdout.flush()
    for path in ["{CSV_ABS_PATH}", "temp/data.csv", "../temp/data.csv", "../../temp/data.csv", "../../../temp/data.csv", "data.csv"]:
        if os.path.exists(path):
            print(f"✅ Found data file: {path}"); sys.stdout.flush()
            return path
    print("❌ No data file found in expected locations. Please ensure 'temp/data.csv' exists."); sys.stdout.flush()
    return None

try:
    print("📊 Starting data analysis..."); sys.stdout.flush()
    pd = ensure_package("pandas"); np = ensure_package("numpy")
    plt = ensure_package("matplotlib", "matplotlib.pyplot"); seaborn = ensure_package("seaborn")
    try:
        import matplotlib; matplotlib.use("Agg")
    except Exception: pass
    try:
        np.random.seed(42); random.seed(42)
    except Exception: pass

    data_file_path = find_data_file()
    if data_file_path is None: sys.exit(1)

    print(f"📂 Loading data from: {data_file_path}"); sys.stdout.flush()
    df = pd.read_csv(data_file_path, encoding="utf-8-sig",
                     dtype={"bank_name":"string","cardholder":"string","description":"string","Category":"string"})

    # Normalize
    if "amount" in df.columns:
        s = df["amount"].astype(str).str.strip()
        s = s.str.replace(r"[,$]", "", regex=True).str.replace(r"\\(", "-", regex=True).str.replace(r"\\)", "", regex=True)
        df["amount"] = pd.to_numeric(s, errors="coerce")
    if "transaction_date" in df.columns:
        df["transaction_date"] = pd.to_datetime(df["transaction_date"], errors="coerce", utc=False)
        if df["transaction_date"].isna().any():
            orig = pd.read_csv(data_file_path, encoding="utf-8-sig", usecols=["transaction_date"])["transaction_date"]
            for fmt in ["%m/%d/%Y","%Y-%m-%d","%d/%m/%Y","%m-%d-%Y","%Y/%m/%d","%B %d, %Y","%b %d, %Y","%d-%b-%Y","%d %B %Y"]:
                na = df["transaction_date"].isna()
                if na.any():
                    df.loc[na, "transaction_date"] = pd.to_datetime(orig[na], format=fmt, errors="coerce")
            unparsed = df["transaction_date"].isna().sum()
            if unparsed>0:
                print(f"⚠️ Could not parse {unparsed} date values"); sys.stdout.flush()
    for col in ["bank_name","cardholder","description","Category"]:
        if col in df.columns:
            df[col] = df[col].fillna("").astype("string").str.strip()

    before=len(df); df=df[df["amount"].notna()]; after=len(df)
    if before>after: print(f"🧹 Dropped {before-after} rows with invalid amounts."); sys.stdout.flush()
    if "transaction_date" in df.columns:
        missing_dates = df["transaction_date"].isna().sum()
        if missing_dates>0: print(f"⚠️ {missing_dates} rows have unparsed dates but valid amounts - keeping them."); sys.stdout.flush()
# ----------------- BOILERPLATE END -----------------
    # <<< YOUR ANALYSIS CODE GOES HERE >>>
    # Build report content in a variable; generate tables and charts; enforce quality gate.
# ----------------- ERROR HANDLING START -----------------
except FileNotFoundError as e:
    print(f"⚠️ File not found error: {e}. Please ensure the data file exists at temp/data.csv."); sys.stdout.flush(); sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {e}"); print(f"Traceback: {traceback.format_exc()}"); sys.stdout.flush(); sys.exit(1)
finally:
    print("✅ Analysis script finished."); sys.stdout.flush()
# ----------------- ERROR HANDLING END -----------------

Key Scripting Rules:

Do not create any directories. Save all files to the current working directory.
All charts must be saved as .png files with the fixed filenames listed above. Do not use pie charts or subplots.
Always embed charts in the markdown by reading the saved PNG via embed_image.
Maintain the generated_images list; after embedding, validate Base64 lengths (>1000). If a chart is too small/invalid, regenerate with simplified settings or a placeholder.
Enforce the Quality Gate: exactly the 6 required slots must exist (use placeholders if needed), and embed at least 6 images before writing the report.
Print a final confirmation: "Report written to report.md (charts embedded: N, tables rendered: M)".
"""

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.anthropic import AnthropicChatCompletionClient
import os
from datetime import datetime, timezone
import uuid

from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import TextMessage
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_agentchat.ui import Console
from dotenv import load_dotenv

load_dotenv()

# Decoding controls (with safe defaults)
TEMPERATURE = float(os.getenv("LLM_TEMPERATURE", "0"))
TOP_P = float(os.getenv("LLM_TOP_P", "1"))
SEED = int(os.getenv("LLM_SEED", "45"))

# Model configuration (left as-is)
ANTHROPIC_MODEL = os.getenv("ANTHROPIC_MODEL", "claude-sonnet-4-20250514")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY2")

# Directory configuration
TEMP_DIR = "temp"
CSV_PATH = os.path.join(TEMP_DIR, "data.csv")
CSV_ABS_PATH = os.path.abspath(CSV_PATH)

# Ensure source CSV exists in temp (do NOT copy it elsewhere)
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"CSV not found at {CSV_PATH}. Please place 'data.csv' in the 'temp' directory.")

# Create a per-message/run working directory under temp and an output/ directory inside it
run_id = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S_") + uuid.uuid4().hex[:8]
WORK_DIR = os.path.join(TEMP_DIR, f"run_{run_id}")
OUTPUT_DIR = os.path.join(WORK_DIR, "output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# System message template (CSV-aware; includes Category column; reads directly from temp/data.csv)
DATA_ANALYZER_SYSTEM_MESSAGE_TEMPLATE = DATA_ANALYZER_SYSTEM_MESSAGE_TEMPLATE

# Inject the absolute CSV path into the system message
DATA_ANALYZER_SYSTEM_MESSAGE = DATA_ANALYZER_SYSTEM_MESSAGE_TEMPLATE.replace("{CSV_ABS_PATH}", CSV_ABS_PATH)

def get_openai_client():
    """Get configured OpenAI model client."""
    return OpenAIChatCompletionClient(
        model=OPENAI_MODEL,
        api_key=OPENAI_API_KEY,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        seed=SEED,
    )

model_client = get_openai_client()

# def get_anthropic_client():
#     """Get configured OpenAI model client."""
#     return AnthropicChatCompletionClient(
#         model=ANTHROPIC_MODEL,
#         api_key=ANTHROPIC_API_KEY,
        # temperature=TEMPERATURE,
        # top_p=TOP_P,
        # seed=SEED,
#     )

# model_client = get_anthropic_client()

# Code executor attached to this run-specific working directory (per-message outputs live under temp/run_*/output)
code_executor = LocalCommandLineCodeExecutor(work_dir=WORK_DIR)

# Agents
data_analyzer_agent = AssistantAgent(
    name="Data_Analyzer",
    model_client=model_client,
    system_message=DATA_ANALYZER_SYSTEM_MESSAGE,
    description="Data analysis agent that processes and analyzes CSV data directly from temp/data.csv."
)

code_executor_agent = CodeExecutorAgent(
    name="Python_Code_Executor",
    code_executor=code_executor,
    description="Python code executor agent that runs Python scripts locally."
)

# Termination condition - stop when "STOP" is mentioned or after max messages
termination_condition = TextMentionTermination("STOP") | MaxMessageTermination(max_messages=30)

# Team setup: start with the analyzer so it can plan and emit code first
generator_team = RoundRobinGroupChat(
    participants=[data_analyzer_agent, code_executor_agent],
    termination_condition=termination_condition,
    max_turns=20  # Reduced from 100
)

# Provide your question here. Adjust as needed.
user_question = (
    "Analyze the CSV and produce a comprehensive markdown report with monthly spend trends by cardholder, "
    "by Category, and top merchants inferred from the description field, including at least two charts."
)
# Example of a specific question:
# user_question = "Which Category has the highest total spend this year, and which cardholder is the top contributor? Include a bar chart."

# Compose the user task message
task = TextMessage(
    content=(
        "You must read the CSV directly from temp/data.csv using the absolute path provided in your system prompt. "
        "Do not move or copy the file. Follow the Execution Protocol and Python Scripting Guidelines.\n\n"
        f"User question: {user_question}\n\n"
        f"Per-run output directory (already created): {OUTPUT_DIR}. "
        "Your script should save into the relative path 'output/' which maps to that directory."
    ),
    source="user"
)

# Run and stream to console (works in Jupyter via top-level await)
generator_result = await Console(generator_team.run_stream(task=task))

print(f"\nRun directory: {WORK_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print("If a report was generated, it should be at: output/report.md relative to the run directory above.")
